In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [29]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
submission = pd.read_csv("sample_submission.tsv")

#subtrain = train.sample(frac=0.2)
cnt = 10000
X = np.array(train.drop(['Num', 'y'], axis=1))
y = np.array(train['y'])
X = X[-cnt:]
y = y[-cnt:]

In [30]:
y_mean = train.y.mean()
print(y_mean)
train.describe()

198575.912031


,Num,y,year,week,shift,item_id,f1,f2,f3,f4,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
count,72457.000000,7.245700e+04,72457.000000,72457.000000,72457.000000,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,...,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04,7.245700e+04
mean,178904.496129,1.985759e+05,2013.524325,26.439116,1.993610,2.044181e+07,1.313229e+05,1.324994e+05,1.326374e+05,1.326679e+05,...,1.277857e+05,1.276425e+05,1.272761e+05,1.264721e+05,1.265909e+05,1.263964e+05,1.260598e+05,1.253287e+05,1.255156e+05,1.252468e+05
std,103879.795910,4.755776e+05,0.541215,15.405824,0.816497,5.875915e+03,3.386385e+05,3.396957e+05,3.394491e+05,3.388524e+05,...,3.120409e+05,3.115069e+05,3.103819e+05,3.073558e+05,3.073772e+05,3.067469e+05,3.055666e+05,3.025618e+05,3.025612e+05,3.019187e+05
min,0.000000,1.000000e+00,2012.000000,1.000000,1.000000,2.042703e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,89711.000000,4.902000e+03,2013.000000,13.000000,1.000000,2.043868e+07,2.320000e+03,2.425000e+03,2.460000e+03,2.490000e+03,...,3.015000e+03,3.040000e+03,3.048000e+03,3.050000e+03,3.070000e+03,3.080000e+03,3.090000e+03,3.080000e+03,3.090000e+03,3.090000e+03
50%,177658.000000,3.271100e+04,2014.000000,26.000000,2.000000,2.044098e+07,1.880000e+04,1.925100e+04,1.932500e+04,1.943000e+04,...,2.029300e+04,2.034000e+04,2.034000e+04,2.030900e+04,2.038000e+04,2.038000e+04,2.038600e+04,2.038100e+04,2.046000e+04,2.046000e+04
75%,270259.000000,1.598350e+05,2014.000000,40.000000,3.000000,2.044394e+07,9.969000e+04,1.013700e+05,1.015560e+05,1.016300e+05,...,1.013250e+05,1.013050e+05,1.009900e+05,1.005220e+05,1.007180e+05,1.005220e+05,1.002340e+05,1.000420e+05,1.003300e+05,1.001210e+05
max,349518.000000,9.881526e+06,2015.000000,53.000000,3.000000,2.045356e+07,6.763986e+06,6.763986e+06,6.763986e+06,6.763986e+06,...,6.137594e+06,6.137594e+06,6.137594e+06,6.137594e+06,6.137594e+06,6.137594e+06,6.137594e+06,6.137594e+06,6.137594e+06,6.137594e+06


In [31]:
from sklearn.model_selection import TimeSeriesSplit

In [33]:
def smape(y1, y2):
    n = len(y1)
    return 200 * (np.abs(y1 - y2) / (np.abs(y1) + np.abs(y2))).sum() / n

def cross_val(model, X, y):
    tscv = TimeSeriesSplit(n_splits=3)
    scores = []
    for train, test in tscv.split(X):
        model.fit(X[train], y[train])
        y_pred = model.predict(X[test])
        
        # plt.plot(np.arange(len(y_pred - y[test])), y_pred - y[test])
        # plt.show()
        
        print("train %.2f, test %.2f" % (smape(y_pred, y[test]), smape(model.predict(X[train]), y[train])))
        
        scores.append(smape(y_pred, y[test]))
    return np.mean(scores)

In [41]:
import xgboost as xgb
from xgboost import XGBRegressor

In [42]:
model = XGBRegressor(max_depth=15, n_estimators=50)

In [43]:
print("smape %.2f" % cross_val(model, X, y))

train 43.22, test 4.47
train 27.32, test 5.39
train 50.15, test 7.16
smape 40.23


In [45]:
model.fit(X, y)
res = model.predict(np.array(test.drop(['Num'], axis=1)))

In [46]:
submission['y'] = res

In [47]:
submission.describe()

,Num,y
count,2016.000000,2.016000e+03
mean,350371.290675,1.329686e+05
std,806.372912,2.910094e+05
min,348622.000000,3.626750e+01
25%,349796.750000,3.509310e+03
50%,350520.500000,2.297804e+04
75%,351024.250000,1.026860e+05
max,351528.000000,2.578912e+06


In [48]:
submission.to_csv("ans.csv", index=False)